<h1> Introduction to metaprogramming in Julia </h1>

## Generating Julia code from within Julia

### David P. Sanders

#### Facultad de Ciencias, Universidad Nacional Autónoma de México (UNAM)
#### Department of Mathematics & Julia Lab, MIT

### "High-level code that writes high-level code" (Steven Johnson)

## What is metaprogramming?

Most of the programs that we write are pieces of code whose purpose is to manipulate data in some way.

Sometimes, though, we need to do **metaprogramming**. 
The word ["**meta**"](https://en.wikipedia.org/wiki/Meta) roughly means "something on a higher level".
So **metaprogramming** means  "higher-level programming": writing code (a program) that manipulates *code*. (And the new code will then manipulate data.)


# Motivation: Why do we need metaprogramming?

The short answer is that we don't *need* it at all! Everything we can do with metaprogramming can be done, in principle, without it.
But metaprogramming can *make our life easier / better*, by automating the generation of repetitive bits of code (often called **boilerplate** code).

Here are two simple examples:

Suppose we have a variable `my_long_variable` and we wish to display the value *and the name* of the variable. We could write

In [1]:
my_long_variable = 3

println("my_long_variable = ", my_long_variable)

my_long_variable = 3


But it's annoying that I need to write `my_long_variable` twice &nbsp; I would like to write it only once.
Unfortunately this is *not possible* using a normal Julia function, since when you pass an argument to a function, you pass its *value*; all information about where that value came from is forgotten.

For example, tt's easy to make a function `myshow` that displays the *value* that a variable is bound to:

In [2]:
function myshow(y)
    println("The value is $y")
end

myshow (generic function with 1 method)

In [5]:
a = 1
b = 2

2

In [6]:
myshow(a + b)

The value is 3


But if we want to know *whose* value is that, the function *does not know* &ndash; it receives *only* the value itself.

The solution is to use a **macro**; macros are indicated with the `@` symbol:

In [70]:
@show y

y = 3


3

In [7]:
@show a + b

a + b = 3


3

We see that the macro in some sense "has access" to the *name* of the variable.

What is happening here? A macro takes a *piece of code* (expression) and modifies it to produce a *new piece of code*. 
The process of producing new code is often called **code generation**.

This new piece of code replaces the old piece of code *before* anything gets compiled, so that the compiler never sees the original code, but only the new code. Later in the workshop we will see how this works in more detail. 
For now, we just remark that it is possible to see exactly what any macro is doing by displaying the new code this macro *expands* to using `@macroexpand` as follows. For now we just want to get a feel for what's happening, not look at the details of the code that is generated:

In [71]:
@macroexpand @show y

quote
    Base.println("y = ", Base.repr(begin
                #= show.jl:955 =#
                local var"#59#value" = y
            end))
    var"#59#value"
end

We can remove the line number information with `Base.remove_linenums!`:

In [77]:
Base.remove_linenums!( @macroexpand @show y )

quote
    Base.println("y = ", Base.repr(begin
                local var"#76#value" = y
            end))
    var"#76#value"
end

Another example is `@time`. This wraps boilerplate (i.e. repetitive) timing code around the given piece of code. Again it needs access to the code itself, not the *result* of running that code:

In [8]:
@time exp(1)

  0.000000 seconds


2.718281828459045

In [9]:
using BenchmarkTools

In [15]:
x = 1.0

1.0

In [16]:
@btime exp($(Ref(x))[])

  10.286 ns (0 allocations: 0 bytes)


2.718281828459045

In [83]:
Base.remove_linenums!( @macroexpand @time exp(1) )

quote
    while false
    end
    local var"#82#stats" = Base.gc_num()
    local var"#85#compile_elapsedtime" = Base.cumulative_compile_time_ns_before()
    local var"#84#elapsedtime" = Base.time_ns()
    local var"#83#val" = exp(1)
    var"#84#elapsedtime" = Base.time_ns() - var"#84#elapsedtime"
    var"#85#compile_elapsedtime" = Base.cumulative_compile_time_ns_after() - var"#85#compile_elapsedtime"
    local var"#86#diff" = Base.GC_Diff(Base.gc_num(), var"#82#stats")
    Base.time_print(var"#84#elapsedtime", (var"#86#diff").allocd, (var"#86#diff").total_time, Base.gc_alloc_count(var"#86#diff"), var"#85#compile_elapsedtime", true)
    var"#83#val"
end

In summary, metaprogramming can reduce the burden on the user by automating the generation of repetitive bits of code.

In [17]:
@edit @time exp(1)

## Metaprogramming for Domain-Specific Languages

Following the same idea, we can also use metaprogramming to automate less mundane tasks, namely the generation of complex scientific models. 

Often a Julia package for defining and simulating scientific models will create new Julia types representing those models, containing necessary information for simulating them.
But the user would like to specify such models in a "domain-specific language", which is much more mathematicalj, and this mathematical syntax needs some pre-processing before 
the corresponding Julia objects can be created.

A great example of this is the [Catalyst.jl](https://github.com/SciML/Catalyst.jl) package for specifying networks of chemical reactions. 
The user-facing syntax 

```
@reaction_network begin
  k1, A --> B
  k2, B --> A
end k1 k2 
```

looks like the specification of two chemical reactions from a chemistry textbook. Using a macro allows us to *sidestep* standard Julia syntax ("spelling") and
*invent our own syntax*. 

Running this code gives the output

```
julia> network = @reaction_network begin
         k1, A --> B
         k2, B --> A
       end k1 k2
       
Model ##ReactionSystem#257 with 2 equations
States (2):
  A(t)
  B(t)
Parameters (2):
  k1
  k2
  

julia> typeof(network)
ReactionSystem
  
  
```




The macro then processes that syntax into actual Julia code and creates a standard Julia object as a result.
Running `@macroexpand` on this gives

```
julia> @macroexpand @reaction_network begin
         k1, A --> B
         k2, B --> A
       end k1 k2
quote
    begin
        var"#66#t" = (ModelingToolkit.toparam)((Symbolics.wrap)((Sym){Real}(:t)))
        var"#67#k1" = (ModelingToolkit.toparam)((Symbolics.wrap)((Sym){Real}(:k1)))
        var"#68#k2" = (ModelingToolkit.toparam)((Symbolics.wrap)((Sym){Real}(:k2)))
        [var"#66#t", var"#67#k1", var"#68#k2"]
    end
    begin
        var"#69#A" = (identity)((Symbolics.wrap)(((Sym){(SymbolicUtils.FnType){Catalyst.NTuple{1, Catalyst.Any}, Real}}(:A))((Symbolics.value)(var"#66#t"))))
        var"#70#B" = (identity)((Symbolics.wrap)(((Sym){(SymbolicUtils.FnType){Catalyst.NTuple{1, Catalyst.Any}, Real}}(:B))((Symbolics.value)(var"#66#t"))))
        [var"#69#A", var"#70#B"]
    end
    Catalyst.ReactionSystem([Catalyst.Reaction(var"#67#k1", [var"#69#A"], [var"#70#B"], [1], [1], only_use_rate = false), Catalyst.Reaction(var"#68#k2", [var"#70#B"], [var"#69#A"], [1], [1], only_use_rate = false)], var"#66#t", [var"#69#A", var"#70#B"], [var"#67#k1", var"#68#k2"]; name = Symbol("##ReactionSystem#258"))
end
```

The macro has done a lot of work to convert the symbolic description into a Julia object!

Note, however, that there is a *limitation* on the syntax allowed inside macros: we *cannot* use any syntax we want in a macro, but only syntax that "obeys the rules of how Julia syntax is parsed". As an example, parentheses must always be balanced in Julia, so it would not be possible to write a macro for half-open intervals that processed `x = (a, b]`. To use *arbitrary* syntax like this we would have to resort instead to [non-standard string literals](https://docs.julialang.org/en/v1/manual/strings/#non-standard-string-literals).

### When should we *not* use metaprogramming?

Metaprogramming allows us to replace pieces of code with different bits of code. For example, given an expression `x * y`, we will see how we can replace the `*` inside the piece of code with `+`. However, Julia is a powerful language that provides many abstractions that mean that it is usually *not* necessary to use metaprogramming. For example, we can do this exact replacement using higher-order functions, without metaprogramming, as follows:

In [32]:
function f(x, y, ⊕)  # ⊕
    return ⊕(x, y)
end

f (generic function with 1 method)

We can now pass in *different* binary operators instead of `+`!:

In [25]:
f(3, 4, +)

7

In [27]:
f(3, 4, *)

12

In [87]:
f(3, 4, max)

4

In [28]:
3 max 4

LoadError: syntax: extra token "max" after end of expression

In [30]:
3 + 4

7

In [31]:
+(3, 4)

7

However, now consider a more complicated piece of code like `x + y - z / (w * t)`. If we wish to replace each of the operators by a different one, it starts to get cumbersome to pass them all as parameters. This is quite a difficult problem in general, which has spawned multiple attempts to solve it in the Julia ecosystem. We will see a couple of possible solutions during this workshop.

## Non-standard string literals

Example:

In [34]:
big"0.1"

0.1000000000000000000000000000000000000000000000000000000000000000000000000000002

In [37]:
@big_str("0.1")

0.1000000000000000000000000000000000000000000000000000000000000000000000000000002

In [38]:
"k, A --> B"

"k, A --> B"

In [39]:
using Catalyst

In [40]:
network = @reaction_network begin
         k1, A --> B
         k2, B --> A
       end k1 k2

Model ##ReactionSystem#300 with 2 equations
States (2):
  A(t)
  B(t)
Parameters (2):
  k1
  k2

In [43]:
network = @reaction_network begin
         k1, A --> B
         k2, B --> A
       end k1

LoadError: UndefVarError: k2 not defined

In [44]:
@time @time exp(1)

  0.000000 seconds
  0.000198 seconds (43 allocations: 1.484 KiB)


2.718281828459045

In [46]:
@time @time sin(1)

  0.000000 seconds
  0.000125 seconds (45 allocations: 1.516 KiB)


0.8414709848078965